In [24]:
import importlib.util
#The default paths for windows
import configuration
spec_win = importlib.util.spec_from_file_location('lumapi', configuration.LUMERICAL_API_PATH)
#Functions that perform the actual loading
lumapi = importlib.util.module_from_spec(spec_win) # 
spec_win.loader.exec_module(lumapi)
import numpy as np

C:\Program Files\Lumerical\v242\api\python\lumapi.py:895: SyntaxWarning: invalid escape sequence '\s'
  message = re.sub('^(Error:)\s(prompt line)\s[0-9]+:', '', str(rvals[2])).strip()


In [25]:
class LumObj: 
    def __init__(self, env) -> None:
        self._env  = env 

    @property
    def env(self):
        return self._env   
    
class GeomBuilder(LumObj):
    
    def add_wg_top(
        self,
        name="InP_top_wg",
        material="InP - Palik",
        width: float = 550e-9,
        height: float = 313e-9,
        length: float = 10e-6,
        x: float = 0,
        y: float = 0,
        z: float = 0,
        x_min: float = None,
        y_min: float = None,
        z_min: float = None,
        layout_group_name="Input Waveguide"
    ):
         
        env = self.env
        env.groupscope('::model') 

        # Set center positions if the minimum values are provided
        if x_min is not None: x = x_min + length / 2
        if y_min is not None: y = y_min + width / 2
        if z_min is not None: z = z_min + height / 2
        
        

        # Create top waveguide
        rect = env.addrect({"name": name})
        
        rect.material = material
        rect.x = x
        rect.y = y
        rect.z = z
        rect.x_span = length
        rect.y_span = width
        rect.z_span = height    
        

        env.addtogroup(layout_group_name)
        env.groupscope('::model')
        
        return rect
    
    def add_wg_bottom(
        self,
        name="SiN_bottom_wg",
        material="Si3N4 (Silicon Nitride) - Phillip",
        width: float = 550e-9,
        height: float = 350e-9,
        length: float = 10e-6,
        x: float = 0,
        y: float = 0,
        z: float = 0,
        x_min: float = None,
        y_min: float = None,
        z_min: float = None,
        layout_group_name="Input Waveguide"
    ):
        env = self.env
        env.groupscope('::model') 

        # Set center positions if the minimum values are provided
        if x_min is not None: x = x_min + length / 2
        if y_min is not None: y = y_min + width / 2
        if z_min is not None: z = z_min + height / 2

        # Create bottom waveguide
        rect = env.addrect({"name": name})
        
        rect.material = material
        rect.x = x
        rect.y = y
        rect.z = z
        rect.x_span = length
        rect.y_span = width
        rect.z_span = height

        env.addtogroup(layout_group_name)
        env.groupscope('::model')
        
        return rect

    def taper_top(
        self,
        name='Taper_top',
        height: float = 250e-9,
        length: float = 19e-6,
        width_in: float = 550e-9,
        width_out: float = 50e-9,
        material: str = 'InP - Palik',
        m: float = 0.8,
        layout_group_name="Taper",
        x=0,
        y=0,
        z=0,
        x_min=None,
        y_min=None,
        z_min=None
    ):
        """
        Create a top taper structure with specified parameters and add it to the simulation environment.

        Parameters
        ----------
        name : str, optional
            Name of the taper structure, defaults to 'Taper_in'.
        height : float, optional
            Height of the taper structure, defaults to 313e-9.
        length : float, optional
            Length of the taper structure, defaults to 19e-6.
        width_in : float, optional
            Input width of the taper structure, defaults to 550e-9.
        width_out : float, optional
            Output width of the taper structure, defaults to 50e-9.
        material : str, optional
            Material of the taper structure, defaults to 'InP - Palik'.
        m : float, optional
            Exponent for the taper profile, defaults to 0.8.
        layout_group_name : str, optional
            Layout group name, defaults to "Taper".
        z : float, optional
            Z-coordinate of the center of the taper, defaults to 313e-9 / 2.
        """
        env = self.env

        # Script for creating the taper
        script = ''' 
        res = 5000; # resolution of polygon
        xspan = linspace(-len/2, len/2, res);
        a = (w1/2 - w2/2) / len^m;
        yspan = a * (len * 0.5 - xspan)^m + w2/2;
        
        V = matrix(2 * res, 2);
        # [x, y] points
        V(1:2 * res, 1) = [xspan, flip(xspan, 1)]; 
        V(1:2 * res, 2) = [-yspan, flip(yspan, 1)];
        
        addpoly;
        set("name", "taper");
        set("x", 0);
        set("y", 0);
        set("z", 0);
        set("z span", h1);
        set("vertices", V);
        set("material", mat);
        '''

        # Create taper structure group
        taper = env.addstructuregroup({'name': name})
        env.select(taper.name)
        env.set('construction group', 1)

        # Add user properties for the taper
        env.adduserprop('len', 2, length)
        env.adduserprop('h1', 2, height)
        env.adduserprop('w1', 2, width_in)
        env.adduserprop('w2', 2, width_out)
        env.adduserprop('m', 0, m)
        env.adduserprop('mat', 1, material)

        # Set the script and position for the taper
        taper.script = script
        
        # Set center positions if the minimum values are provided
        if x_min is not None: x = x_min + length / 2
        if y_min is not None: y = y_min + width_in / 2
        if z_min is not None: z = z_min + height / 2
        
        taper.x = x
        taper.y = y
        taper.z = z

        env.select(taper.name)
        env.addtogroup(layout_group_name)
        # Go back to the model group
        env.groupscope('::model')
        
        return taper
    
    def taper_bottom(
        self,
        name='Taper_out',
        height: float = 0.35e-6,
        length: float = 19e-6,
        width_in: float = 550e-9,
        width_out: float = 1.1e-6,
        m: float = 7,
        material: str = 'Si3N4 (Silicon Nitride) - Phillip',
        layout_group_name="Taper",
        x=0,
        y=0,
        z=0,
        x_min=None,
        y_min=None,
        z_min=None
    ):
        """
        Create a bottom taper structure with specified parameters and add it to the simulation environment.

        Parameters
        ----------
        name : str, optional
            Name of the taper structure, defaults to 'Taper_out'.
        height : float, optional
            Height of the taper structure, defaults to 0.35e-6.
        length : float, optional
            Length of the taper structure, defaults to 19e-6.
        width_in : float, optional
            Input width of the taper structure, defaults to 550e-9.
        width_out : float, optional
            Output width of the taper structure, defaults to 1.1e-6.
        m : float, optional
            Exponent for the taper profile, defaults to 7.
        material : str, optional
            Material of the taper structure, defaults to 'Si3N4 (Silicon Nitride) - Phillip'.
        layout_group_name : str, optional
            Layout group name, defaults to "Taper".
        z : float, optional
            Z-coordinate of the center of the taper, defaults to -350e-9 / 2.
        """
        env = self.env

        # Script for creating the taper
        script_taper_out = '''        
        res = 5000; # resolution of polygon
        xspan = linspace(-len/2, len/2, res);
        a = (w1/2 - w2/2) / len^m;
        yspan = a * (len * 0.5 - xspan)^m + w2/2;
        
        V = matrix(2 * res, 2);
        # [x, y] points
        V(1:2 * res, 1) = [xspan, flip(xspan, 1)]; 
        V(1:2 * res, 2) = [-yspan, flip(yspan, 1)];
        
        addpoly;
        set("name", "taper");
        set("x", 0);
        set("y", 0);
        set("z", 0);
        set("z span", h1);
        set("vertices", V);
        set("material", mat);
        '''

        # Create taper structure group
        taper = env.addstructuregroup({'name': name})
        env.select(taper.name)
        env.set('construction group', 1)

        # Add user properties for the taper
        env.adduserprop('len', 2, length)
        env.adduserprop('h1', 2, height)
        env.adduserprop('w1', 2, width_in)
        env.adduserprop('w2', 2, width_out)
        env.adduserprop('m', 0, m)
        env.adduserprop('mat', 1, material)
        
        # Set the script and position for the taper
        taper.script = script_taper_out
        
        # Set center positions if the minimum values are provided
        if x_min is not None: x = x_min + length / 2
        if y_min is not None: y = y_min + width_in / 2
        if z_min is not None: z = z_min + height / 2

        taper.x = x
        taper.y = y
        taper.z = z

        env.select(taper.name)
        env.addtogroup(layout_group_name)
        # Go back to the model group
        env.groupscope('::model')

        return taper
    

    def build_top(
        self,
        x: float = 0,
        y: float = 0,
        z: float = 0,
        z_min: float = None,
        x_min: float = None,
        y_min: float = None,
        input_length: float = 10e-6,
        taper_length: float = 19e-6,
        output_length: float = 0,
        input_width: float = 50e-9,
        output_width: float = 500e-9,
        material: str = 'InP - Palik',
        layout_group_name="Top Waveguide", 
        height: float = 250e-9, 
        m: float = 0.8
    ):   
        total_length = input_length + taper_length + output_length
        input_wg = self.add_wg_top(
            name="top_wg_input",
            material=material,
            width=input_width,
            height=height,
            length=input_length,
            layout_group_name=layout_group_name,
            x_min=-total_length/2,
        )

        taper = self.taper_top(
            name='taper_top',
            height=height,
            length=taper_length,
            width_in=input_width,
            width_out=output_width,
            material=material,
            layout_group_name=layout_group_name,
            m=m,
            x_min=input_length - total_length/2,
        )

        output_wg = self.add_wg_top(
            name="top_wg_output",
            material=material,
            width=output_width,
            height=height,
            length=output_length,
            layout_group_name=layout_group_name,
            x_min=input_length + taper_length - total_length/2,
        )

        self.env.groupscope(f'::model')
        self.env.select(layout_group_name)
        

        if z_min is not None: z = z_min + height / 2
        self.env.set('z', z) 

        
        if x_min is not None: x = x_min + total_length / 2
        self.env.set('x', x)

        max_width = np.max([input_width, output_width])
        if y_min is not None: y = y_min + max_width / 2
        self.env.set('y', y)


        

        self.env.groupscope(f'::model') 

        top_wg = {
            "input_wg": input_wg,
            "taper": taper,
            "output_wg": output_wg
        }

        return top_wg

        

        
    def build_bottom(
        self,
        x: float = 0,
        y: float = 0,
        z: float = 0,
        x_min: float = None,
        y_min: float = None,
        z_min: float = None,
        input_length: float = 10e-6,
        taper_length: float = 19e-6,
        output_length: float = 10e-6,
        input_width: float = 50e-9,
        output_width: float = 500e-9,
        material: str = 'Si3N4 (Silicon Nitride) - Phillip',
        layout_group_name="Bottom Waveguide", 
        height: float = 350e-9, 
        m: float = 7
    ):
        total_length = input_length + taper_length + output_length
        input_wg = self.add_wg_bottom(
            name="bottom_wg_input",
            material=material,
            width=input_width,
            height=height,
            length=input_length,
            layout_group_name=layout_group_name,
            x_min=-total_length/2,
        )

        taper = self.taper_bottom(
            name='taper_bottom',
            height=height,
            length=taper_length,
            width_in=input_width,
            width_out=output_width,
            material=material,
            layout_group_name=layout_group_name,
            m=m,
            x_min=-total_length/2 +input_length,
        )

        output_wg = self.add_wg_bottom(
            name="bottom_wg_output",
            material=material,
            width=output_width,
            height=height,
            length=output_length,
            layout_group_name=layout_group_name,
            x_min=-total_length/2 + input_length + taper_length,
        )

        self.env.groupscope(f'::model')
        self.env.select(layout_group_name)


        if z_min is not None: z = z_min + height / 2
        self.env.set('z', z)

        
        if x_min is not None: x = x_min + total_length / 2
        self.env.set('x', x)

        max_width = np.max([input_width, output_width])
        if y_min is not None: y = y_min + max_width / 2
        self.env.set('y', y)
        
        self.env.groupscope(f'::model') 

        bottom_wg = {
            "input_wg": input_wg,
            "taper": taper,
            "output_wg": output_wg
        }

        return bottom_wg
    

class TaperDesigner:

    def __init__(
        self,
        env,
        height_top=250e-9,
        height_bottom=800e-9,
        height_oxide_layer=100e-9,
        width_top_in=500e-9,
        width_top_out=50e-9,
        width_bottom_in=1e-6,
        width_bottom_out=1e-6,
        length_input=10e-6,
        length_taper=19e-6,
        length_output=10e-6,
        length_output_top=0,
        m_top = 0.8,
        m_bottom = 7,
    ):
        self.env = env
        
        
        self.height_top = height_top
        self.height_bottom = height_bottom
        self.width_top_in = width_top_in
        self.width_top_out = width_top_out
        self.width_bottom_in = width_bottom_in
        self.width_bottom_out = width_bottom_out
        self.length_input = length_input
        self.length_taper = length_taper
        self.length_output = length_output
        self.length_output_top = length_output_top
        self.height_oxide_layer = height_oxide_layer
        self.m_top = m_top
        self.m_bottom = m_bottom
        self.total_height = height_top + height_bottom + height_oxide_layer
        self.total_lenght = length_input + length_taper + length_output
        self.max_width = np.max([width_top_out, width_bottom_out, width_bottom_in, width_top_in])

        self.geom_builder = GeomBuilder(env)
        self.waveguide_top = self.geom_builder.build_top(
            z_min=0,
            x_min=0,
            y=0,
            input_length=self.length_input,
            taper_length=self.length_taper,
            output_length=self.length_output_top,
            input_width=self.width_top_in,
            output_width=self.width_top_out,
            material='InP - Palik',
            layout_group_name="Top Waveguide",
            height=self.height_top,
            m=self.m_top
        )

        self.waveguide_bottom = self.geom_builder.build_bottom(
            z_min=-self.height_oxide_layer-self.height_bottom,
            x_min=0,
            y=0,
            input_length=self.length_input,
            taper_length=self.length_taper,
            output_length=self.length_output,
            input_width=self.width_bottom_in,
            output_width=self.width_bottom_out,
            material='Si3N4 (Silicon Nitride) - Phillip',
            layout_group_name="Bottom Waveguide",
            height=self.height_bottom,
            m=self.m_bottom
        )

    def build_simulation_region(
        self,
        mul_w = 4, 
        mul_h = 4,
        cell_number = 30,
        lam0 = 1.55e-6,
        background_material = 'SiO2 (Glass) - Palik',
        N_modes = 50,
        x_pen = 0,

    ):  
        self.env.groupscope('::model')
        w_EME = mul_w * np.max([self.width_top_out, 
                                self.width_bottom_out, 
                                self.width_bottom_in,
                                self.width_top_in])
        h_EME = mul_h * self.height_bottom + self.height_top + self.height_oxide_layer
        
        # Create the simulation region
        eme = self.env.addeme()
        eme.wavelength = lam0
        eme.x_min  = self.length_input - x_pen
        eme.y = 0
        eme.y_span = w_EME
        eme.z = (self.height_top - self.height_oxide_layer - self.height_bottom)/ 2
        eme.z_span = h_EME

        # Boundary conditions
        eme.y_min_bc = 'PML'
        eme.y_max_bc = 'PML'
        eme.z_min_bc = 'PML'
        eme.z_max_bc = 'PML'
        

        # # Material 
       
        eme.background_material =  background_material

        # Cells
       
        eme.number_of_cell_groups = 3
        eme.group_spans = np.array([x_pen, self.length_taper, x_pen])
        eme.cells = np.array([1, cell_number, 1])
        eme.subcell_method = np.array([0, 1, 0])  # 0 = None, 1 = CVCS
        eme.display_cells = 1  
        eme.number_of_modes_for_all_cell_groups = N_modes
        
        self.env.groupscope('::model')
        self.simulation_region = eme

        
        return eme
    

    def build_mesh(self, 
                   mul_w_mesh: float = 1.5, 
                   mul_h_mesh: float = 1.5, 
                   dx: float = 5e-9, 
                   dy: float = 5e-9, 
                   dz: float = 0.01e-6):
        """
        Build the mesh for the simulation environment.

        Args:
            mul_w_mesh (float, optional): Multiplier for the width of the mesh. Defaults to 1.5.
            mul_h_mesh (float, optional): Multiplier for the height of the mesh. Defaults to 1.5.
            dx (float, optional): Mesh grid size in the x-direction. Defaults to 5e-9.
            dy (float, optional): Mesh grid size in the y-direction. Defaults to 5e-9.
            dz (float, optional): Mesh grid size in the z-direction. Defaults to 0.01e-6.
        """

        self.env.groupscope('::model')

        w_mesh = mul_w_mesh * self.max_width
        h_mesh = mul_h_mesh * self.total_height
        len_mesh = self.total_lenght

        # Override mesh
        mesh = self.env.addmesh()

        mesh.y = 0
        mesh.y_span = w_mesh
        mesh.z = (self.height_top -self.height_bottom - self.height_oxide_layer) / 2
        mesh.z_span = h_mesh
        mesh.x = (self.length_input + self.length_taper + self.length_output) / 2
        mesh.x_span = len_mesh
        mesh.dx = dx
        mesh.dy = dy
        mesh.dz = dz

        self.mesh = mesh

        return mesh

       
        
    def build_monitors(self):
        """
        Build the monitors for the simulation environment.

        Args:
            mul_w (float, optional): Multiplier for the width of the EME region. Defaults to 3.
            mul_h_mesh (float, optional): Multiplier for the height of the mesh. Defaults to 1.5.
        """
        

        # InP
        field_monitor_top = self.env.addemeprofile({"name": "monitor_field_top"})
        field_monitor_top.y = 0
        field_monitor_top.y_span = self.simulation_region.y_span
        field_monitor_top.z = self.height_top / 2
        field_monitor_top.x = self.simulation_region.x
        field_monitor_top.x_span = self.mesh.x_span


        index_monitor_top = self.env.addemeindex({"name": "monitor_index_top"})
        index_monitor_top.y = 0
        index_monitor_top.y_span = self.simulation_region.y_span
        index_monitor_top.z = self.height_top / 2
        index_monitor_top.x = self.simulation_region.x
        index_monitor_top.x_span = self.mesh.x_span

        # SiN
        field_monitor_bottom = self.env.addemeprofile({"name": "monitor_field_bottom"})
        field_monitor_bottom.y = 0
        field_monitor_bottom.y_span = self.simulation_region.y_span
        field_monitor_bottom.z = (-self.height_oxide_layer - self.height_bottom)/ 2
        field_monitor_bottom.x = self.simulation_region.x
        field_monitor_bottom.x_span = self.mesh.x_span

        index_monitor_bottom = self.env.addemeindex({"name": "monitor_index_bottom"})
        index_monitor_bottom.y = 0
        index_monitor_bottom.y_span = self.simulation_region.y_span
        index_monitor_bottom.z = (-self.height_oxide_layer - self.height_bottom) / 2
        index_monitor_bottom.x = self.simulation_region.x
        index_monitor_bottom.x_span = self.mesh.x_span

        # Y=0

        field_monitor_y0 = self.env.addemeprofile({"name": "monitor_field_y0"})
        field_monitor_y0.monitor_type = "2D Y-normal" 
        field_monitor_y0.y = 0
        field_monitor_y0.z = 0
        field_monitor_y0.z_span = self.simulation_region.z_span
        field_monitor_y0.x = self.simulation_region.x
        field_monitor_y0.x_span = self.simulation_region.x_span

        index_monitor_y0 = self.env.addemeindex({"name": "monitor_index_y0"})
        index_monitor_y0.monitor_type = "2D Y-normal"
        index_monitor_y0.y = 0
        index_monitor_y0.z = 0
        index_monitor_y0.z_span = self.simulation_region.z_span
        index_monitor_y0.x = self.simulation_region.x
        index_monitor_y0.x_span = self.simulation_region.x_span
       

        self.field_monitors = {
            "top": field_monitor_top,
            "bottom": field_monitor_bottom,
            "y0": field_monitor_y0
        }

        self.index_monitors = {
            "top": index_monitor_top,
            "bottom": index_monitor_bottom,
            "y0": index_monitor_y0
        }

        return self.field_monitors, self.index_monitors


        



In [26]:
env = lumapi.MODE()


In [27]:
name = "shape_sweep_"
env.deleteall()
taper_designer = TaperDesigner(env)
length_input = taper_designer.length_input
eme  = taper_designer.build_simulation_region(x_pen = 5/10*length_input)
mesh = taper_designer.build_mesh()
field_monitors, index_monitors = taper_designer.build_monitors()  


In [28]:
env = lumapi.MODE()
name = "shape_sweep_"
m_top = np.linspace(0.8, 1.8, 10)
env.deleteall()

for i in range(len(m_top)):
    taper_designer = TaperDesigner(env, m_top = m_top[i])
    taper_designer.build_simulation_region(x_pen = 5/10*length_input)
    taper_designer.build_mesh()
    taper_designer.build_monitors()
    env.save(f'{name}{i}.fsp')
    env.deleteall()

env.close()